## Single Models

Description : 

This notebook contains final version of all the final used in the paper. The list of models tested are : 
- Lasso
- Random Forest 
- MLP
- LSTM

In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Seed for reproductibility
SEED = 123

import numpy.random
np.random.seed(SEED)

import random
random.seed(SEED)

# Remove warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
houses = [6,40,59,72,87,60] # ID of house selected in the EWHDataset.csv file, respectively called 1-6 in the paper

In [162]:
# Train Test Split

def TrainTest(house_number,output='Demand'):
    """
    output is either Demand, Binary_Consumption, Percentage_Consumption or Quantile_Consumption
    """
    X = pd.read_csv(f'Data/house_{house_number}_Features.csv',index_col=0)
    y = pd.read_csv(f'Data/house_{house_number}_Target.csv',index_col=0)
    X.index=pd.to_datetime(X.index)
    y.index=pd.to_datetime(y.index)
    
    X_train = X.loc[:'2018-02']
    X_test = X.loc['2018-03':]
    
    y_train = y.loc[:'2018-02',[output]]
    y_test = y.loc['2018-03':,[output]]
        
    return X_train,y_train,X_test,y_test

### Lasso

In [163]:
# Parameters
alpha = 0.01

In [164]:
for idx, house in enumerate(houses):
    
    X_train,y_train,X_test,y_test = TrainTest(house,output='Demand')

    reg = Lasso(alpha=alpha,random_state=SEED)

    reg.fit(X_train,y_train)

    y_preds = reg.predict(X_test)
    y_preds[y_preds<0] = 0
    
    # R-coef
    R_coef = np.corrcoef(y_test.values.reshape(-1),
            y_preds)[1,0]
    
    # RMSE
    RMSE = np.sqrt(mean_squared_error(y_test.values.reshape(-1),
                           y_preds))
    print(30*'-')
    print(f'House {house} - id {idx+1}')
    print(30*'-')
    print(f'\tR : {R_coef}')
    print(f'\tRMSE : {RMSE}')
    print()

------------------------------
House 6 - id 1
------------------------------
	R : 0.6526682265335282
	RMSE : 6.813566514596529

------------------------------
House 40 - id 2
------------------------------
	R : 0.5323713520841594
	RMSE : 9.913585114669061

------------------------------
House 59 - id 3
------------------------------
	R : 0.5921777081794859
	RMSE : 16.59869403987528

------------------------------
House 72 - id 4
------------------------------
	R : 0.6508593506672528
	RMSE : 15.39163303390047

------------------------------
House 87 - id 5
------------------------------
	R : 0.580348782514308
	RMSE : 10.052256647326004

------------------------------
House 60 - id 6
------------------------------
	R : 0.202894105038354
	RMSE : 2.360875129953985



### Random Forest

In [166]:
for idx, house in enumerate(houses):
    
    X_train,y_train,X_test,y_test = TrainTest(house,output='Demand')

    reg = RandomForestRegressor(n_estimators = 100, 
                            min_samples_split = 17,
                            random_state=SEED)

    reg.fit(X_train,y_train)

    y_preds = reg.predict(X_test)
    y_preds[y_preds<0] = 0
    
    # R-coef
    R_coef = np.corrcoef(y_test.values.reshape(-1),
            y_preds)[1,0]
    
    # RMSE
    RMSE = np.sqrt(mean_squared_error(y_test.values.reshape(-1),
                           y_preds))
    print(30*'-')
    print(f'House {house} - id {idx+1}')
    print(30*'-')
    print(f'\tR : {R_coef}')
    print(f'\tRMSE : {RMSE}')
    print()

------------------------------
House 6 - id 1
------------------------------
	R : 0.6678953088355168
	RMSE : 6.652424047311198

------------------------------
House 40 - id 2
------------------------------
	R : 0.615884811571865
	RMSE : 9.1997517562241

------------------------------
House 59 - id 3
------------------------------
	R : 0.6318793382747818
	RMSE : 15.890173008137804

------------------------------
House 72 - id 4
------------------------------
	R : 0.7157153692955447
	RMSE : 14.080521157695205

------------------------------
House 87 - id 5
------------------------------
	R : 0.5604796613588767
	RMSE : 10.22579114799964

------------------------------
House 60 - id 6
------------------------------
	R : 0.22905152773738222
	RMSE : 2.382023945378234



### MLP

In [167]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1_l2

In [168]:
for idx, house in enumerate(houses):
    
    X_train,y_train,X_test,y_test = TrainTest(house,output='Demand')

    # Define model
    inputs = Input(shape=(19,))
    
    # a layer instance is callable on a tensor, and returns a tensor
    x = Dense(64, activation='relu',kernel_regularizer=l1_l2(l1=1e-2, l2=1e-2))(inputs)
    x = Dense(32, activation='relu',kernel_regularizer=l1_l2(l1=1e-2, l2=1e-2))(inputs)
    regression = Dense(1, activation='relu')(x)
    
    model = Model(inputs=inputs, outputs=regression)
    model.compile(optimizer='adam',
                loss='mean_squared_error')
    
    # Train model 
    model.fit(X_train, y_train,epochs=256,verbose=False)
    
    # Predict Model
    y_preds = model.predict(X_test)
    
    y_preds[y_preds<0] = 0
        
    # R-coef
    R_coef = np.corrcoef(y_test.values.reshape(-1),
                y_preds.reshape(-1))[1,0]
        
    # RMSE
    RMSE = np.sqrt(mean_squared_error(y_test.values.reshape(-1),
                               y_preds.reshape(-1)))
    print(30*'-')
    print(f'House {house} - id {idx+1}')
    print(30*'-')
    print(f'\tR : {R_coef}')
    print(f'\tRMSE : {RMSE}')
    print()

------------------------------
House 6 - id 1
------------------------------
	R : 0.666243746086888
	RMSE : 6.667856030978653

------------------------------
House 40 - id 2
------------------------------
	R : 0.5727945163029785
	RMSE : 9.562262509714623

------------------------------
House 59 - id 3
------------------------------
	R : 0.6053267423618656
	RMSE : 16.330382014309954

------------------------------
House 72 - id 4
------------------------------
	R : 0.6508354560804553
	RMSE : 15.462568361512782

------------------------------
House 87 - id 5
------------------------------
	R : 0.5632112401975102
	RMSE : 10.233933150625175

------------------------------
House 60 - id 6
------------------------------
	R : 0.09194483741102816
	RMSE : 2.4675084522490973



### LSTM

In [169]:
from tensorflow.keras.layers import LSTM

In [170]:
for idx, house in enumerate(houses):
    
    X_train,y_train,X_test,y_test = TrainTest(house,output='Demand')
    
    n_steps = 1
    n_features = X_train.shape[1]

    # Define model
    inputs = Input(shape = (n_steps, n_features),name = 'input')

    x = LSTM(16, activation='relu', input_shape=(n_steps, n_features))(inputs)

    output = (Dense(1, name = 'output',activation='relu'))(x)

    model = Model(inputs = [inputs], outputs = output )
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    #model.summary()
    
    # Train model 
    model.fit(X_train.values.T.reshape(X_train.shape[0],1,X_train.shape[1]), y_train,epochs=64,verbose=False)
        
    # Predict Model
    y_preds = model.predict(X_test.values.T.reshape(X_test.shape[0],1,X_test.shape[1]))
    
    y_preds[y_preds<0] = 0
        
    # R-coef
    R_coef = np.corrcoef(y_test.values.reshape(-1),
                y_preds.reshape(-1))[1,0]
        
    # RMSE
    RMSE = np.sqrt(mean_squared_error(y_test.values.reshape(-1),
                               y_preds.reshape(-1)))
    print(30*'-')
    print(f'House {house} - id {idx+1}')
    print(30*'-')
    print(f'\tR : {R_coef}')
    print(f'\tRMSE : {RMSE}')
    print()

------------------------------
House 6 - id 1
------------------------------
	R : 0.11789905733197771
	RMSE : 10.823757805168027

------------------------------
House 40 - id 2
------------------------------
	R : 0.1108259588247078
	RMSE : 12.865823202952276

------------------------------
House 59 - id 3
------------------------------
	R : 0.04646389784268899
	RMSE : 22.314302180183557

------------------------------
House 72 - id 4
------------------------------
	R : 0.10232236502326493
	RMSE : 21.31491985418901

------------------------------
House 87 - id 5
------------------------------
	R : 0.09199223840363828
	RMSE : 12.971740681574332

------------------------------
House 60 - id 6
------------------------------
	R : -0.019700369758182706
	RMSE : 2.581305327250367

